In [ ]:
import os
import cv2
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from sklearn.utils import shuffle

In [ ]:
dataset_path = "histology_slides"  
regex_label = r"[A-Z]+_([A-Z])_[A-Z]+-\d{2}-[A-Z\d]+-(\d+)-\d+\.png"

In [ ]:
def load_data(dataset_path, img_size=(128,128)):

    X = []
    y = []

    num_files = 0

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.png'):

                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size)
                X.append(img)

                # assumindo que label seja True ou False para maligno
                match_obj = re.search(regex_label, file)
                label = match_obj.group(1) # B(enigno) ou M(aligno)
                y.append(True if label == "M" else False)

                num_files = num_files + 1
                if num_files%500 == 0 and num_files != 0:
                    print(str(num_files) + ": " + img_path)
                    X = np.array(X)
                    y = np.array(y)
                    return X, y

    print(num_files)

    X = np.array(X)
    y = np.array(y)
    return X, y

In [ ]:
img_size = (700, 460)
X, y = load_data(dataset_path, img_size)

In [ ]:
X = X / 255.0

In [ ]:
X, y = shuffle(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def build_cnn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
cnn_model = build_cnn(X_train.shape[1:])
cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
cnn_predictions = np.argmax(cnn_model.predict(X_test), axis=-1)
cnn_precision = precision_score(y_test, cnn_predictions, average='weighted')

In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_flat, y_train)
knn_predictions = knn.predict(X_test_flat)
knn_accuracy = accuracy_score(y_test, knn_predictions)
knn_precision = precision_score(y_test, knn_predictions, average='weighted')

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_flat, y_train)
rf_predictions = rf.predict(X_test_flat)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions, average='weighted')

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_flat, y_train)
lr_predictions = lr.predict(X_test_flat)
lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_precision = precision_score(y_test, lr_predictions, average='weighted')

In [ ]:
nb = GaussianNB()
nb.fit(X_train_flat, y_train)
nb_predictions = nb.predict(X_test_flat)
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_precision = precision_score(y_test, nb_predictions, average='weighted')

In [ ]:
print(f"CNN - Accuracy: {cnn_accuracy:.4f}, Precision: {cnn_precision:.4f}")
print(f"KNN - Accuracy: {knn_accuracy:.4f}, Precision: {knn_precision:.4f}")
print(f"Random Forest - Accuracy: {rf_accuracy:.4f}, Precision: {rf_precision:.4f}")
print(f"Logistic Regression - Accuracy: {lr_accuracy:.4f}, Precision: {lr_precision:.4f}")
print(f"Naive Bayes - Accuracy: {nb_accuracy:.4f}, Precision: {nb_precision:.4f}")